In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, welch, find_peaks
import glob
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split

In [2]:
# For plotting pandas dataframes
def plot_df(df):
    # Plotting each column with row number as x-axis
    df.plot()

    # Adding labels and title
    plt.xlabel('Row Number')
    plt.ylabel('Values')
    plt.title('Pandas DataFrame Plot')
    plt.show()

def plot_ndarray(arr):
    plt.figure(figsize=(10, 5))
    plt.plot(arr, label='Values')
    plt.legend()
    plt.title('Autocorrelation')
    plt.xlabel('Lag')
    plt.ylabel('Autocorrelation value')
    plt.show()

In [3]:
# Global variables

# 2D Array, 53 features per row (after adjustments to features)
all_features = np.empty((0, 53))
# 1D array, 1 number per row
all_answers = list()
# List of every file name in RawData
file_names = glob.glob("RawData" + '/*.txt')

In [4]:
def psd(data, str):
    # Calculate the Power Spectral Density (PSD) using Welch's method
    frequencies, psd = welch(data, fs=50, nperseg=len(data))

    # Find peaks
    peaks, _ = find_peaks(psd)

    # Extract first 3 peaks by height
    sorted_peaks = np.argsort(psd[peaks])[-3:]
    first_3_peaks = peaks[sorted_peaks]
    first_3_peak_values = psd[first_3_peaks]
    
    # # Plotting
    # plt.figure(figsize=(12, 6))
    # plt.semilogy(frequencies, psd, label='PSD')
    # plt.plot(frequencies[first_3_peaks], first_3_peak_values, 'r^', label='Peaks')
    # plt.title(f'Power Spectral Density (PSD) with Peaks of {str}')
    # plt.xlabel('Frequency [Hz]')
    # plt.ylabel('Power/Frequency [dB/Hz]')
    # plt.grid(True)
    # plt.legend()
    # plt.show()

    # # Print the first 3 peaks and valleys along with their frequencies
    # print(frequencies[first_3_peaks])
    # print(first_3_peak_values)
    # print("First 3 Peaks:")
    # for i, (freq, val) in enumerate(zip(frequencies[first_3_peaks], first_3_peak_values), 1):
    #     print(f"Peak {i}: Frequency = {freq:.2f} Hz, Value = {val:.2f} dB/Hz")

    # frequencies[first_3_peaks] is location (x-value of a graph), first_3_peak_values is like a y-value
    return (frequencies[first_3_peaks], first_3_peak_values)



In [5]:
def extractWindow(acc_data, gyro_data, activityNum: int):
    cur_features_row = list()

    # Calculate mean for each direction in acc and gyro (6 total)
    mean_acc_x = acc_data[0].mean()
    mean_acc_y = acc_data[1].mean()
    mean_acc_z = acc_data[2].mean()
    mean_gyro_x = gyro_data[0].mean()
    mean_gyro_y = gyro_data[1].mean()
    mean_gyro_z = gyro_data[2].mean()
    means = [mean_acc_x, mean_acc_y, mean_acc_z, mean_gyro_x, mean_gyro_y, mean_gyro_z]
    for mean in means:
        cur_features_row.append(mean)

    # Calculate RMS for all (6 total numbers)
    rms_acc_x = np.sqrt(np.mean(acc_data[0]**2))
    rms_acc_y = np.sqrt(np.mean(acc_data[1]**2))
    rms_acc_z = np.sqrt(np.mean(acc_data[2]**2))
    rms_gyro_x = np.sqrt(np.mean(gyro_data[0]**2))
    rms_gyro_y = np.sqrt(np.mean(gyro_data[1]**2))
    rms_gyro_z = np.sqrt(np.mean(gyro_data[2]**2))
    rmses = [rms_acc_x, rms_acc_y, rms_acc_z, rms_gyro_x, rms_gyro_y, rms_gyro_z]
    for rms in rmses:
        cur_features_row.append(rms)

#========================================================================================================
    # # No more autocorrelation
    # composite_signal_series = pd.Series(acc_data[0])
    # var = composite_signal_series.var()
    # print(var)


    # # Calculate autocorrelation for different lags
    # lags = range(-100, 100)
    # autocorr_values = [composite_signal_series.autocorr(lag=k) for k in lags]


    # # Plotting the autocorrelation on the existing plot
    # plt.figure(figsize=(10, 6))
    # plt.plot(lags, autocorr_values, marker='o', label=activityNum)
    # plt.axhline(0, color='black', linewidth=0.5)
    # plt.title('Autocorrelation Comparison')
    # plt.xlabel('Time lag (s)')
    # plt.ylabel('Correlation')
    # plt.grid(True)
    # plt.legend(loc='best')
    # plt.show()
#=================================================================================================================

    # Spectral peaks (36)
    multi_tuples = list()
    multi_tuples.append(psd(acc_data[0], "Acc_X"))
    multi_tuples.append(psd(acc_data[1], "Acc_Y"))
    multi_tuples.append(psd(acc_data[2], "Acc_Z"))
    multi_tuples.append(psd(gyro_data[0], "Gyro_X"))
    multi_tuples.append(psd(gyro_data[1], "Gyro_Y"))
    multi_tuples.append(psd(gyro_data[2], "Gyro_Z"))

    # Adds all 3 positions of the peak, then all 3 values of the peak
    for tuple in multi_tuples:
        for arr in tuple:
            for data_pt in arr:
                cur_features_row.append(data_pt)

    # Resultant/magnitude acceleration (1) 
    acc_mag = np.sqrt(mean_acc_x**2 + mean_acc_y**2 + mean_acc_z**2)
    cur_features_row.append(acc_mag)

    # Resultant gyro (1)
    gyro_mag = np.sqrt(mean_gyro_x**2 + mean_gyro_y**2 + mean_gyro_z**2)
    cur_features_row.append(gyro_mag)

    # Angle btwn resultant acc and each acc input (3)
    angle_accX = np.degrees(np.arccos(mean_acc_x / acc_mag))
    angle_accY = np.degrees(np.arccos(mean_acc_y / acc_mag))
    angle_accZ = np.degrees(np.arccos(mean_acc_z / acc_mag))
    cur_features_row.append(angle_accX)
    cur_features_row.append(angle_accY)
    cur_features_row.append(angle_accZ)


    cur_features_row_as_npArr = np.array(cur_features_row)

    global all_features
    all_features = np.vstack((all_features, cur_features_row_as_npArr))
    all_answers.append(activityNum)

In [6]:
'''These are for noise reduction'''
# Function to apply median filter
def apply_median_filter(data, window_size):
    return data.rolling(window=window_size, center=True, min_periods=1).median()

# Function to design a low-pass Butterworth filter
def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs  # Nyquist frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

# Function to apply the Butterworth filter
def apply_low_butter(data, cutoff, fs, order):
    b, a = butter_lowpass(cutoff, fs, order)
    y = filtfilt(b, a, data, axis=0)
    return y

'''This is for getting body acc component from total acc'''
# Define a function to create a high-pass Butterworth filter
def butter_highpass(cutoff, fs, order):
    nyquist = 0.5 * fs  # Nyquist frequency is half the sampling rate
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

# Define a function to apply the high-pass filter to data
def apply_high_butter(data, cutoff, fs, order):
    b, a = butter_highpass(cutoff, fs, order)
    y = filtfilt(b, a, data, axis=0)
    return y

In [7]:
def extractAllFeatures(label: str):
    splitted = label.split()
    experimentNum, activityNum, start, end = int(splitted[0]), int(splitted[2]), int(splitted[3]), int(splitted[4])

    acc_file_name = file_names[experimentNum-1]
    gyro_file_name = file_names[experimentNum-1+61]

    # Reading acc and gyro in data for this label range
    acc_data = pd.read_csv(acc_file_name, sep=' ', header=None, skiprows=start, nrows=end-start)
    gyro_data = pd.read_csv(gyro_file_name, sep=' ', header=None, skiprows=start, nrows=end-start)

    ''' I'm not too sure how these filters work but I hope they do'''
    # Apply median filter to each axis of accelerometer and gyroscope data (for noise reduction)
    WINDOW_SIZE = 3
    acc_data_filtered = acc_data.apply(apply_median_filter, window_size=WINDOW_SIZE)
    gyro_data_filtered = gyro_data.apply(apply_median_filter, window_size=WINDOW_SIZE)
    
    # Apply low-pass Butterworth filter to each axis of accelerometer and gyroscope data (for noise reduction)
    FS = 50                     # Sampling frequency in Hz (you mentioned 50Hz)
    CUTOFF_LOW_PASS = 20        # Desired cutoff frequency of the filter, Hz
    ORDER = 3                   # Order of the Butterworth filter
    acc_data_filtered = acc_data_filtered.apply(lambda col: apply_low_butter(col, CUTOFF_LOW_PASS, FS, ORDER))
    gyro_data_filtered = gyro_data_filtered.apply(lambda col: apply_low_butter(col, CUTOFF_LOW_PASS, FS, ORDER))    

    # Apply high-pass Butterworth filter to acc to get body component
    CUTOFF_HIGH_PASS = 0.3
    acc_data_filtered = acc_data_filtered.apply(lambda col: apply_low_butter(col, CUTOFF_HIGH_PASS, FS, ORDER))

    # 2.56sec windows * 50 samples per sec = 128 samples per window
    # 50% overlap means we go up by 64 for every new window
    # end-64 is used to skip the last half window that gets added
    length = end-start
    i = 0
    while i < length-64:
        trueEnd = min(i+128, length)
        extractWindow(acc_data_filtered[i: trueEnd], gyro_data_filtered[i: trueEnd], activityNum)
        i += 64


'''For testing'''
# extractAllFeatures('1 1 5 250 1232')
# extractAllFeatures('1 1 7 1233 1392')
# extractAllFeatures('1 1 4 1393 2194')
# extractAllFeatures('1 1 1 7496 8078')
# extractAllFeatures('1 1 2 14069 14699')
# extractAllFeatures('1 1 3 14869 15492')

# extractAllFeatures('1 1 1 7496 8078')
# extractAllFeatures('1 1 2 14069 14699')
# extractAllFeatures('1 1 3 14869 15492')

'For testing'

In [8]:
# Main function
def main():
    # Grab the labels.txt file telling you what each data section means
    labels_file = open('labels.txt') 
    labels_list = labels_file.readlines()

    # For each section: 
        # i. Load that section's respective data
        # ii. For each window in that section:
            # a. Apply noise filters on the window
            # b. Compute features from the window
            # c. Store those feature values in an array
    for label in labels_list:
        extractAllFeatures(label)
    
    '''I checked: all_features and all_answers are the same length (same number of rows)'''

    # Take all_features and answers, split into test and train, and finally write to .txt file
    X_train, X_test, y_train, y_test = train_test_split(all_features, all_answers, test_size=0.3, random_state=42)
    
    np.savetxt("X_train.txt", X_train, delimiter=' ', fmt='%.8f')
    np.savetxt("X_test.txt", X_test, delimiter=' ', fmt='%.8f')
    np.savetxt("y_train.txt", y_train, delimiter=' ', fmt='%d')
    np.savetxt("y_test.txt", y_test, delimiter=' ', fmt='%d')
    
    

if __name__ == "__main__":
    main()

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 53 and the array at index 1 has size 35